In [1]:
using SQLite
import SQLite.Stmt
import SQLite.DBInterface.execute
using Distributions

In [2]:
db1 = SQLite.DB("output/before/sweep_db_gathered.sqlite")
db2 = SQLite.DB("output/after/sweep_db_gathered.sqlite")

SQLite.DB("output/after/sweep_db_gathered.sqlite")

In [3]:
[(run_id, time, value) for (run_id, time, value) in execute(db2, "SELECT * FROM run_meta WHERE key = \"elapsed_time\"")]

15-element Vector{Tuple{Int64, String, Float64}}:
 (1, "elapsed_time", 1135.237)
 (2, "elapsed_time", 1158.01)
 (3, "elapsed_time", 1156.333)
 (4, "elapsed_time", 1162.495)
 (5, "elapsed_time", 1161.778)
 (6, "elapsed_time", 1158.812)
 (7, "elapsed_time", 1168.615)
 (8, "elapsed_time", 1160.55)
 (9, "elapsed_time", 1170.501)
 (10, "elapsed_time", 1163.236)
 (11, "elapsed_time", 1171.181)
 (12, "elapsed_time", 1179.154)
 (13, "elapsed_time", 1178.85)
 (14, "elapsed_time", 1161.194)
 (15, "elapsed_time", 1157.89)

In [4]:
import StatsAPI.pvalue
import StatsBase.mean
import StatsBase.mad
import HypothesisTests.ApproximateTwoSampleKSTest

# Functions to compare using K-S tests

"""
    Do K-S tests using SQLite queries that take one parameter, run_id.

    `db1` and `db2` are assumed to have identical parameter combinations labeled the same way,
    via the `combo_id` column in the `runs` table.

    The query should return a single value for the provided run_id.

    If values from db2 need to be extracted with a different query, provide a value for `q2`.
"""
function compare(db1, db2, q1; q2 = q1)
    combo_ids = get_combo_ids(db1)
    @assert all(combo_ids .== get_combo_ids(db2))
    [(combo_id, compare(db1, db2, combo_id, q1; q2 = q2)) for combo_id in combo_ids]
end

function get_combo_ids(db)
    [combo_id for (combo_id,) in execute(db, "SELECT DISTINCT combo_id FROM runs ORDER BY combo_id")]
end

function get_run_ids(db, combo_id)
    [run_id for (run_id,) in execute(db, "SELECT run_id FROM runs WHERE combo_id = ?", [combo_id])]
end

function compare(db1, db2, combo_id, q1; q2 = q1)
    v1 = query_values_for_combo_id(db1, combo_id, q1)
    mean1 = mean(v1)
    mad1 = mad(v1; center = mean1)

    v2 = query_values_for_combo_id(db2, combo_id, q2)
    mean2 = mean(v2)
    mad2 = mad(v2; center = mean2)

    (ApproximateTwoSampleKSTest(v1, v2), (mean1, mad1), (mean2, mad2))
end

function query_values_for_combo_id(db, combo_id, q)
    stmt = Stmt(db, q)
    run_ids = get_run_ids(db, combo_id)

    [query_value_for_run_id(stmt, run_id) for run_id in run_ids]
end

function query_value_for_run_id(stmt, run_id)
    for (value,) in execute(stmt, (run_id,))
        return value
    end
end

query_value_for_run_id (generic function with 1 method)

In [5]:
for (combo_id, (test, (mean1, mad1), (mean2, mad2))) in compare(
    db1, db2, "SELECT value FROM run_meta WHERE key = \"elapsed_time\" AND run_id = ?"
)
    println("Testing elapsed time for parameter combination $(combo_id)...")
    println("    (mean1 $(mean1), mad1 $(mad1))")
    println("    (mean2 $(mean2), mad2 $(mad2))")
    println("    p-value: $(pvalue(test))")
    if pvalue(test) < 0.01
        println("    DIFFERENT distributions w/ p < 0.01")
    else
        println("    undetectable difference between distributions w/ p < 0.01")
    end
    println("    fractional difference: $((mean1 - mean2) / mean1)")
end

Testing elapsed time for parameter combination 1...
    (mean1 1258.7128666666665, mad1 11.147488400647653)
    (mean2 1162.9224000000002, mad2 7.461047404407668)
    p-value: 6.118046410036549e-7
    DIFFERENT distributions w/ p < 0.01
    fractional difference: 0.07610192062335824
